In [ ]:
Pkg.update()
Pkg.add("Jacobi")
Pkg.add("FastGaussQuadrature")
Pkg.add("PyPlot")

In [ ]:
using FastGaussQuadrature
using Jacobi

nodePoints=200 # Number of quadrature nodes used to compute L matrix.

function b_n(n::Int64)
    numerator = -n*(n+1)
    denominator = (n+2)*(n+3)
    return numerator/denominator
end

function M_ij(size::Int64)
    fullMatrix = zeros(size,size)
    for i=0:size-1
        for j=0:size-1
            if i==j
                fullMatrix[i+1, j+1] = 2/(2*i+1) + (b_n(i)*b_n(i))*(2/(2*i+5))
            elseif i==j+2
                fullMatrix[i+1, j+1] = b_n(j)*(2/(2*j+5))
            elseif i==j-2
                fullMatrix[i+1, j+1] = b_n(i)*(2/(2*j+1))
            end
        end
    end
    return fullMatrix/2
end

function B_ij(size::Int64)
    fullMatrix = zeros(size,size)
    for i=0:size-1
        for j=0:size-1
            if i==j
                fullMatrix[i+1,j+1]=2/(2*i+1)
            elseif i==j-2
                fullMatrix[i+1,j+1]=b_n(i)*2/(2*i+5)
            end
        end
    end
    return fullMatrix/2
end

function L_ij(size::Int64, S::Function)
    fullArray = zeros(size,size)
    values, weights = gausslegendre(nodePoints)
    for i=0:size-1
        for j=i:size-1
            sumTotal = 0
            for k=0:nodePoints-1
                dp_i = 2.*(djacobi(values[k+1],i,0,0)+b_n(i)*djacobi(values[k+1],i+2,0,0))
                dp_j = 2.*(djacobi(values[k+1],j,0,0)+b_n(j)*djacobi(values[k+1],j+2,0,0))
                newPiece = weights[k+1]*S((values[k+1]+1.)/2.)*dp_i*dp_j
                sumTotal = sumTotal + newPiece
            end
            fullArray[i+1,j+1] = .5*sumTotal
            fullArray[j+1,i+1] = .5*sumTotal
        end
    end
    return fullArray
end

function galerkin_p_psi_print(size::Int64, coefficients, value)
    sumTotal = 0
    for i=size-1:-1:0
        p = jacobi(2.*value -1., i, 0,0)+b_n(i)*jacobi(2.*value -1.,i+2,0,0)
        sumTotal = sumTotal + coefficients[i+1]*p
    end
    return sumTotal
end
;

# Linear Stability

In [ ]:
function dq_galerkin_coefficients(size::Int64, dq_bar::Function)
    fullArray = zeros(size)
    values, weights = gausslegendre(nodePoints)
    for i =0:size-1
        sumTotal = 0
        for k=0:nodePoints-1
            p_n = jacobi(values[k+1], i,0,0)
            sumTotal = sumTotal + weights[k+1]*p_n*dq_bar((values[k+1]+1.)/2.)
        end
        fullArray[i+1] = sumTotal*(2*i+1)/2
    end
    return fullArray
end

function dq_galerkin(size::Int64, coefficients, value::Float64)
    sumTotal = 0
    for i=0:size-1
        sumTotal = sumTotal + coefficients[i+1]*jacobi(2.*value-1.,i,0,0)
    end
    return sumTotal
end

function p_psi_function(size::Int64, value::Float64)
    fullArray = zeros(size)
    for i=0:size-1
        fullArray[i+1] = jacobi(2.*value-1.,i,0,0)+b_n(i)*jacobi(2.*value-1.,i+2,0,0)
    end
    return fullArray
end

function p_q_function(size::Int64, value::Float64) # We shouldn't need this function
    fullArray = zeros(size)
    for i=0:size-1
        fullArray[i+1] = jacobi(2.*value-1.,i,0,0)
    end
    return fullArray
end

function dy_vartheta_bar(S::Function, du_bar::Function, value::Float64)
    return -S(value)*du_bar(value)
end

function dy_psi(u_bar::Function, value::Float64)
    return -u_bar(value)
end

function dy_psi_galerkin(size::Int64, u_bar_coefficients, value::Float64)
    return -u_galerkin(size, u_bar_coefficients, value)
end

function u_galerkin_coefficients(size::Int64, dq_coefficients, dvt_plus, dvt_minus, first_u_coefficient, du_bar::Function, S::Function, LMatrix, BMatrix)
    r_piece1 = BMatrix*dq_coefficients
    r_piece2 = dvt_plus*p_psi_function(size,1.0)
    r_piece3 = dvt_minus*p_psi_function(size,0.0)
    rhs = r_piece1-r_piece2+r_piece3
    lhs = LMatrix[2:size,2:size]
    rhs = rhs[2:size]
    coefficients = lhs \ rhs
    return prepend!(coefficients,[first_u_coefficient])
end

function u_galerkin(size::Int64, coefficients, value)
    sumTotal = 0
    for i=0:size-1
        sumTotal = sumTotal + coefficients[i+1]*(jacobi(2.*value -1., i, 0,0)+b_n(i)*jacobi(2.*value -1.,i+2,0,0))
    end
    return sumTotal
end

In [ ]:
function U_ij(size::Int64, u_galerkin_coefficients)
    fullArray = zeros(size,size)
    values, weights = gausslegendre(nodePoints)
    for i=0:size-1
        for j=0:size-1
            sumTotal = 0
            for k=0:nodePoints-1
                p_psi = jacobi(values[k+1],i,0,0)+b_n(i)*jacobi(values[k+1],i+2,0,0)
                p_q = jacobi(values[k+1],j,0,0)
                u_galerk = u_galerkin(size, u_galerkin_coefficients, (values[k+1]+1.)/2.)
                sumTotal = sumTotal + weights[k+1]*p_psi*p_q*u_galerk
            end
            fullArray[i+1, j+1] = .5*sumTotal
        end
    end
    return fullArray
end

function Q_ij(size::Int64, dq_galerkin_coefficients)
    fullArray = zeros(size,size)
    values, weights = gausslegendre(nodePoints)
    for i=0:size-1
        for j=i:size-1
            sumTotal = 0
            for k=0:nodePoints-1
                dq_galerk = dq_galerkin(size, dq_galerkin_coefficients, (values[k+1]+1.)/2.)
                p_psi_i = jacobi(values[k+1],i,0,0)+b_n(i)*jacobi(values[k+1],i+2,0,0)
                p_psi_j = jacobi(values[k+1],j,0,0)+b_n(j)*jacobi(values[k+1],j+2,0,0)
                sumTotal = sumTotal + weights[k+1]*dq_galerk*p_psi_i*p_psi_j
            end
            fullArray[i+1, j+1] = .5*sumTotal
            fullArray[j+1, i+1] = .5*sumTotal
        end
    end
    return fullArray
end

In [ ]:
function psi(size, kx, ky, M_matrix, L_matrix, value)
    first_piece = (kx^2+ky^2)*M_matrix+L_matrix
    second_piece = p_psi_function(size, value)
    return first_piece \ second_piece
end

function computeMatricesForStabilityAnalysis(sizeOfMatrix, S_function, dq_function, du_function,dvt_plus, dvt_minus, first_coefficient)
    B = B_ij(sizeOfMatrix)
    L = L_ij(sizeOfMatrix, S_function)
    M = M_ij(sizeOfMatrix)
    dq_coefficients = dq_galerkin_coefficients(sizeOfMatrix, dq_function)
    u_coefficients = u_galerkin_coefficients(sizeOfMatrix, dq_coefficients, dvt_plus, dvt_minus, first_coefficient, du_function, S_function, L, B)

    U = U_ij(sizeOfMatrix, u_coefficients)
    Q = Q_ij(sizeOfMatrix, dq_coefficients)
    #println("Matrices Computed")
    return (B,L,M,U,Q, u_coefficients, dq_coefficients)
end

function getSimplifiedStabilityEigenObjectsOfKx(sizeOfMatrix, beta_value, ky, kxValues, S_function, dq_function, du_function, dvt_plus, dvt_minus, first_coefficient)
    (B,L,M,U,Q,u_coefficients,dq_coefficients) = computeMatricesForStabilityAnalysis(sizeOfMatrix,S_function, dq_function, du_function, dvt_plus, dvt_minus, first_coefficient)
    #Should have stable eigenvalues
    (values, vectors) = eig(U, B)
    if(minimum(imag(values))>0)
        println("Stable piece is unstable...")
    end
    
    eigenObjects = []
    for i=1:length(kxValues)
        kx = kxValues[i]
        ul = u_galerkin(sizeOfMatrix, u_coefficients, 1.)+dvt_plus*vecdot(p_psi_function(sizeOfMatrix, 1.),psi(sizeOfMatrix,kx,ky,M,L,1.))
        ur = -dvt_plus*vecdot(p_psi_function(sizeOfMatrix, 1.),psi(sizeOfMatrix,kx,ky,M,L,0.))
        ll = dvt_minus*vecdot(p_psi_function(sizeOfMatrix, 0.),psi(sizeOfMatrix,kx,ky,M,L,1.))
        lr = u_galerkin(sizeOfMatrix, u_coefficients, 0.)-dvt_minus*vecdot(p_psi_function(sizeOfMatrix, 0.),psi(sizeOfMatrix,kx,ky,M,L,0.))
        lhs = [[ul ur]; [ll lr]]
        eigenResult = eigfact(lhs)
        append!(eigenObjects, [eigenResult])
    end
    #println("Done computing eigenvalues and eigenvectors for all kx's")
    return eigenObjects
end

function getCompleteStabilityEigenObjectsOfKx(sizeOfMatrix, beta_value, ky, kxValues, S_function, dq_function, du_function, dvt_plus, dvt_minus, first_coefficient)
    (B,L,M,U,Q,u_coefficients,dq_coefficients) = computeMatricesForStabilityAnalysis(sizeOfMatrix,S_function, dq_function, du_function, dvt_plus, dvt_minus, first_coefficient)
    eigenObjects = []

    for i=1:length(kxValues)
        kx = kxValues[i]
        psi_plus = psi(sizeOfMatrix,kx,ky,M,L,1.)
        psi_minus = psi(sizeOfMatrix,kx,ky,M,L,0.)
        p_psi_plus = p_psi_function(sizeOfMatrix, 1.)
        p_psi_minus = p_psi_function(sizeOfMatrix, 0.)
        centerRowMatrix = ((kx^2+ky^2)*M+L)\B
        
        ul = u_galerkin(sizeOfMatrix, u_coefficients, 1.)+dvt_plus*vecdot(p_psi_plus,psi_plus)
        uc = -(dvt_plus)*(p_psi_plus'*centerRowMatrix)
        ur = -dvt_plus*vecdot(p_psi_plus, psi_minus)
        cl = (Q+beta_value*M)*psi_plus
        cc = U-(Q+beta_value*M)*centerRowMatrix
        cr = -(Q+beta_value*M)*psi_minus
        ll = dvt_minus*vecdot(p_psi_minus,psi_plus)
        lc = -(dvt_minus)*(p_psi_minus'*centerRowMatrix)
        lr = u_galerkin(sizeOfMatrix, u_coefficients, 0.)-dvt_minus*vecdot(p_psi_minus,psi_minus)
        lhs = [[ul uc ur];[cl cc cr];[ll lc lr]]
        lhs_small = [[ul ur]; [ll lr]]
        
        rhs = [[1 zeros(1,sizeOfMatrix) 0];[zeros(sizeOfMatrix,1) B zeros(sizeOfMatrix,1)]; [0 zeros(1,sizeOfMatrix) 1]]
        eigenResult = eigfact(lhs, rhs)
        append!(eigenObjects, [eigenResult])
    end
    #println("Done computing eigenvalues and eigenvectors for all kx's")
    return eigenObjects
end
    
function getStabilityValues(eigenObjectsArray, kxValues)
    steps = length(kxValues)
    growthRates = zeros(steps)
    waveSpeeds = zeros(steps)
    minWaveSpeeds = zeros(steps)
    eigenvectors = []
    
    for i=1:steps
        kx = kxValues[i]
        max_im, max_ind = findmax(imag(eigenObjectsArray[i][:values]))
        append!(eigenvectors, [eigenObjectsArray[i][:vectors][:,max_ind]])
        waveSpeeds[i] = real(eigenObjectsArray[i][:values][max_ind])
        minWaveSpeeds[i] = minimum(real(eigenObjectsArray[i][:values]))
        growthRates[i] = max_im*kx
    end
    
    return growthRates, eigenvectors, waveSpeeds, minWaveSpeeds
end

function createFourStabilityPlots(sizeOfMatrix, S_function, growthRates, eigenvectors, waveSpeeds, minWaveSpeeds, kx_values, ky_value)
    B = B_ij(sizeOfMatrix)
    L = L_ij(sizeOfMatrix, S_function)
    M = M_ij(sizeOfMatrix)
    
    maxIndex = indmax(growthRates)
    maxKx = kx_values[maxIndex]
    #println("Max growth rate at kx=$maxKx")

    highVector = eigenvectors[maxIndex]
    vectorSize = length(highVector)
    highVector_bottom = highVector[vectorSize] 
    psi_coeffs = highVector[1]*psi(sizeOfMatrix, maxKx, ky_value, M, L, 1.)-highVector_bottom*psi(sizeOfMatrix,maxKx,ky_value, M,L,0.)
    psi_coeffs = psi_coeffs - ( (maxKx^2 + ky_value^2)*M + L ) \ B*highVector[2:end-1]
    
    heights = linspace(0,1,100)
    psi_angles = zeros(length(heights))
    psi_amps = zeros(length(heights))
    for i=1:length(heights)
        psi_val = galerkin_p_psi_print(sizeOfMatrix, psi_coeffs, heights[i])
        psi_angles[i] = angle(psi_val)
        psi_amps[i] = abs(psi_val)
    end

    #print(kxs[25])
    fig, axes = subplots(2,2)
    ax = axes[1,1]
    ax[:plot](kx_values, growthRates, label="N = $matrixSize")
    ax[:legend](loc="upper right")
    ax = axes[2,1]
    ax[:plot](kx_values, w_speeds, "b-")
    #ax[:plot](kx_values, w_minSpeeds, "b-")
    
    ax = axes[1,2]
    ax[:plot](psi_angles - psi_angles[1], heights)

    ax = axes[2,2]
    ax[:plot](psi_amps/maximum(psi_amps), heights)
end

# Eady Problem

In [ ]:
function z_linear(value::Float64)
    return value
end

function dz(value::Float64)
    return 1
end 

function dq(value::Float64)
    return 0
end

function S_constant(value::Float64)
    return 1.
end

In [ ]:
using PyPlot

#Analytic Solution
function c(kxValue::Float64)
    inner = (kxValue/2-coth(kxValue/2))*(kxValue/2-tanh(kxValue/2))
    c_value = 1/2 + 1/kxValue*((Complex(inner))^(1/2))
    return c_value
end

function AB(kxValue::Float64)
    c_value = c(kxValue)
    matrix = [ 1 kxValue*c_value; (c_value-1)*kxValue*sinh(kxValue)+1*cosh(kxValue) (c_value-1)*kxValue*cosh(kxValue)+1*sinh(kxValue)]
    solutions = nullspace(matrix)
    return solutions
end

function waveFunction(kxValue::Float64, zHeight)
    ABVal = AB(kxValue)
    waveReturn = ABVal[1]*cosh(kxValue*zHeight)+ABVal[2]*sinh(kxValue*zHeight)
    return waveReturn
end

function createFourStabilityPlotsCompareToExact(sizeOfMatrix, S_function, growthRates, eigenvectors, waveSpeeds, kx_values, ky_value)
    B = B_ij(sizeOfMatrix)
    L = L_ij(sizeOfMatrix, S_function)
    M = M_ij(sizeOfMatrix)
    
    maxIndex = indmax(growthRates)
    maxKx = kx_values[maxIndex]
    highVector = eigenvectors[maxIndex]
    vectorSize = length(highVector)
    highVector_bottom = highVector[vectorSize] 
    psi_coeffs = highVector[1]*psi(sizeOfMatrix, maxKx, ky_value, M, L, 1.)-highVector_bottom*psi(sizeOfMatrix,maxKx,ky_value, M,L,0.)
    psi_coeffs = psi_coeffs - ( (maxKx^2 + ky_value^2)*M + L ) \ B*highVector[2:end-1]
    
    heights = linspace(0,1,100)
    psi_angles = zeros(length(heights))
    psi_amps = zeros(length(heights))
    psi_angles_exact = zeros(length(heights))
    psi_amps_exact = zeros(length(heights))
    for i=1:length(heights)
        psi_val = galerkin_p_psi_print(sizeOfMatrix, psi_coeffs, heights[i])
        psi_angles[i] = angle(psi_val)
        psi_amps[i] = abs(psi_val)
        
        psiValueExact = waveFunction(1.61, heights[i])
        psi_angles_exact[i]= angle(psiValueExact)
        psi_amps_exact[i] = abs(psiValueExact)
    end

    #Exact Growth Rates and wave speeds
    growthRates_exact = zeros(length(kx_values))
    w_speeds_exact = zeros(length(kx_values))
    for i =1:length(kx_values)
        c_value = c(kx_values[i])
        growthRates_exact[i] = kx_values[i]*imag(c_value)
        w_speeds_exact[i] = real(c_value)
    end
    
    fig, axes = subplots(2,2)
    ax = axes[1,1]
    ax[:plot](kx_values, growthRates, label="Galerkin")
    ax[:plot](kx_values, growthRates_exact, label="Exact")
    ax[:legend](loc="upper right")
    ax = axes[2,1]
    ax[:plot](kx_values, w_speeds, label="Galerkin")
    ax[:plot](kx_values, w_speeds_exact, label="Exact")
    ax[:set_ylim](0,1)
    ax[:legend](loc="lower right")
    
    ax = axes[1,2]
    ax[:plot](psi_angles - psi_angles[1], heights, label="Galerkin")
    ax[:plot](psi_angles_exact - psi_angles_exact[1], heights, label="Exact")
    ax[:legend](loc="upper left")
    ax = axes[2,2]
    ax[:plot](psi_amps/maximum(psi_amps), heights, label="Galerkin")
    ax[:plot](psi_amps_exact/maximum(psi_amps_exact), heights, label="Exact")
    ax[:legend](loc="center right")
end

In [ ]:
#Testing galerkin u
#=
first_coefficient = 0.5
matrixSize = 15
B = B_ij(matrixSize)
L = L_ij(matrixSize, S_constant)
M = M_ij(matrixSize)

dqc = dq_galerkin_coefficients(matrixSize, dq)
ugc = u_galerkin_coefficients(matrixSize, dqc,dy_vartheta_bar(S_constant, dz, 1.), dy_vartheta_bar(S_constant, dz, 0.), first_coefficient, dz, S_constant, L, B)

#Test method
u_galerkin(matrixSize, ugc, .1)
#U_ij(matixSize, ugc)

#dq_galerkin(matrixSize, dqc, .3)
#Q_ij(matrixSize, dqc)=#

In [ ]:
using PyPlot
ioff()

nodePoints = 200 # redefining for ease of alteration
matrixSize = 7

first_coefficient = 0.5
kxStart = 0.1
kxFinish = 10.
steps = 100
beta = 0
ky = 0
kxs = linspace(kxStart, kxFinish, steps) 
eigenValuesAndVectors = getCompleteStabilityEigenObjectsOfKx(matrixSize, beta, ky, kxs, S_constant, dq, dz, dy_vartheta_bar(S_constant, dz, 1.), dy_vartheta_bar(S_constant, dz, 0.), first_coefficient)
growthRates, vectors, w_speeds, w_minSpeeds = getStabilityValues(eigenValuesAndVectors, kxs)

createFourStabilityPlotsCompareToExact(matrixSize, S_constant, growthRates, vectors, w_speeds, kxs, ky)

In [ ]:
kxs = 1.61
eigenValuesAndVectors = getCompleteStabilityEigenObjectsOfKx(matrixSize, beta, ky, kxs, S_constant, dq, dz, dy_vartheta_bar(S_constant, dz, 1.), dy_vartheta_bar(S_constant, dz, 0.), first_coefficient)
growthRates, vectors, w_speeds, w_minSpeeds = getStabilityValues(eigenValuesAndVectors, kxs)

# Eady Verification Test

In [ ]:
using PyPlot
ioff()

matrixSize = 5
first_coefficient = 0.5

kxStart = 0.1
kxFinish = 10.
steps = 100
beta = 0
ky = 0
kxs = linspace(kxStart, kxFinish, steps)
eigenValuesAndVectors = getCompleteStabilityEigenObjectsOfKx(matrixSize, beta, ky, kxs, S_constant, dq, dz, dy_vartheta_bar(S_constant, dz, 1.), dy_vartheta_bar(S_constant, dz, 0.), first_coefficient)
growthRates, vectors, w_speeds, w_minSpeeds = getStabilityValues(eigenValuesAndVectors, kxs)

createFourStabilityPlots(matrixSize, S_constant, growthRates, vectors, w_speeds, w_minSpeeds, kxs, ky)

# Ocean-Charney Stability

In [ ]:
function u_OC(value)
    if(value isa Number)
        return 1/54*(3*exp(6*value -6)*(6*value -1) -2 -exp(-6))
    else
        vector = zeros(length(value))
        for i=1:length(value)
            vector[i] = 1/54*(3*exp(6*value[i] -6)*(6*value[i] -1) -2 +exp(-6))
        end
        return vector
    end
end

function dq_OC(value)  
    if(value isa Number)
        return -2
    else
        vector = zeros(length(value))
        for i=1:length(value)
            vector[i] = -2
        end
        return vector
    end
end

function S_OC(value)
    if(value isa Number)
        return exp(6-6*value)
    else
        vector = zeros(length(value))
        for i=1:length(value)
            vector[i] = exp(6-6*value[i])
        end
        return vector
    end
end
   
function du_OC(value)
    if(value isa Number)
        return 2*value*exp(6*(value-1))
    else
        vector = zeros(length(value))
        for i=1:length(value)
            vector[i] = 2*value[i]*exp(6*(value[i]-1))
        end
        return vector
    end
end

In [ ]:
using PyPlot

first_coefficient = 0
matrixSize = 16

kxStart = 0.1
kxFinish = 20.
steps = 200
beta = 1
ky = 0
kxs = linspace(kxStart, kxFinish, steps)

#= Test set 
(B,L,M,U,Q,u_coefficients,dq_coefficients) = computeMatricesForStabilityAnalysis(matrixSize,S_OC, dq_OC, du_OC, -2, 0, first_coefficient)
testPoint = 0.9
realValue = u_OC(testPoint)
galerkinValue = u_galerkin(matrixSize, u_coefficients, testPoint)
x = 0.1:0.01:0.9
plot(x, u_OC(x), x, u_galerkin(matrixSize, u_coefficients, x))
=#

eigenValuesAndVectors = getCompleteStabilityEigenObjectsOfKx(matrixSize, beta, ky, kxs, S_OC, dq_OC, du_OC, -2, 0, first_coefficient)
growthRates, vectors, w_speeds, w_minSpeeds = getStabilityValues(eigenValuesAndVectors, kxs)

matrixSize = 32
eigenValuesAndVectors_2 = getCompleteStabilityEigenObjectsOfKx(matrixSize, beta, ky, kxs, S_OC, dq_OC, du_OC, -2, 0, first_coefficient)
growthRates_2, vectors, w_speeds, w_minSpeeds = getStabilityValues(eigenValuesAndVectors_2, kxs)
println("Done with 16")
matrixSize = 64
eigenValuesAndVectors_3 = getCompleteStabilityEigenObjectsOfKx(matrixSize, beta, ky, kxs, S_OC, dq_OC, du_OC, -2, 0, first_coefficient)
growthRates_3, vectors, w_speeds, w_minSpeeds = getStabilityValues(eigenValuesAndVectors_3, kxs)

fig, axes = subplots(1,3)
ax = axes[1]
ax[:plot](kxs, growthRates, label="N=16")
ax[:legend](loc="upper right")
ax = axes[2]
ax[:plot](kxs, growthRates_2, label="N=32")
ax[:legend](loc="upper right")
ax = axes[3]
ax[:plot](kxs, growthRates_3, label="N=64")
ax[:legend](loc="upper right")
#createFourStabilityPlots(matrixSize, S_OC, growthRates, vectors, w_speeds, w_minSpeeds, kxs, ky)

# Phillips Stability

In [ ]:
function u_Phillips(value)
    if(value isa Number)
        return -cos(pi*value)/pi
    else
        vector = zeros(length(value))
        for i=1:length(value)
            vector[i] = -cos(pi*value[i])/pi
        end
        return vector
    end
end

function dq_Phillips(value)  
    if(value isa Number)
        return pi*cos(pi*value)
    else
        vector = zeros(length(value))
        for i=1:length(value)
            vector[i] = pi*cos(pi*value[i])
        end
        return vector
    end
end

function S_Phillips(value)
    if(value isa Number)
        return 1
    else
        vector = zeros(length(value))
        for i=1:length(value)
            vector[i] = 1
        end
        return vector
    end
end
   
function du_Phillips(value)
    if(value isa Number)
        return sin(pi*value)
    else
        vector = zeros(length(value))
        for i=1:length(value)
            vector[i] = sin(pi*value[i])
        end
        return vector
    end
end

In [ ]:
using PyPlot
first_coefficient = 0
matrixSize = 50

kxStart = 0.1
kxFinish = 4.
steps = 400
beta = 3.1
ky = 0
kxs = linspace(kxStart, kxFinish, steps)

eigenValuesAndVectors = getCompleteStabilityEigenObjectsOfKx(matrixSize, beta, ky, kxs, S_Phillips, dq_Phillips, du_Phillips, 0, 0, first_coefficient)
growthRates_50, vectors, w_speeds, w_minSpeeds = getStabilityValues(eigenValuesAndVectors, kxs)

plot(kxs, growthRates_50, label="Growth Rates at N = 50")
legend(loc="upper right")

In [ ]:

#createFourStabilityPlots(matrixSize, S_Phillips, growthRates, vectors, w_speeds, w_minSpeeds, kxs, ky)
nValues = 5:30
bestValue = findmax(growthRates)
growthRates_forN = zeros(length(nValues))
for i =1:length(nValues)
    kxValue = 3.0
    matrixSize = nValues[i]
    println("Working on $matrixSize")
    eigenValuesAndVectors = getCompleteStabilityEigenObjectsOfKx(matrixSize, beta, ky, kxValue, S_Phillips, dq_Phillips, du_Phillips, 0, 0, first_coefficient)
    growthRate, vectors, w_speeds, w_minSpeeds = getStabilityValues(eigenValuesAndVectors, kxValue)
    growthRates_forN[i] = growthRate[1]
end

semilogy(nValues, abs.(growthRates_forN-bestValue[1]), label="Growth Rate Approach")
legend(loc="upper right")


# First Baroclinic Mode Comparison

In [ ]:
using PyPlot

function isNonZero(value)
    return (value != 0)
end

function firstBaroclinic(value)
    return cos.(pi*value)
end

function computeFirstBaroclinicError(matrixSize, trueFirstFuction, stepsZeroToOne, S_function)
    #Compute Matricies
    M = M_ij(matrixSize)
    #spy(Hermitian(M))
    L = L_ij(matrixSize, S_function)
    G = chol(M)'

    #Compute eigenvalues and eigenvectors
    LGL = (G\L)/(G')
    eigenObject = eigfact(LGL)
    e_vectors = eigenObject[:vectors]
    e_values = eigenObject[:values]
    minimumNonZero = minimum(filter!(isNonZero,e_values))
    index = findfirst(e_values, minimumNonZero)
    neededEVector = e_vectors[:,index]
    neededEVector = (LGL - e_values[index]*eye(matrixSize))\neededEVector
    neededEVector = G'\neededEVector
    #println(neededEVector)
#    normalizationFactor = 1./galerkin_p_psi_print(matrixSize, neededEVector, 0.)
    
    #Calculate Error
    z, weights = gausslegendre(stepsZeroToOne)
    real = trueFirstFuction(z)
    estimate = galerkin_p_psi_print(matrixSize, neededEVector, z)
    normalizationFactor = dot(weights,real.*estimate)/dot(weights,estimate.*estimate)
    estimate = normalizationFactor*estimate
    #println("True Values: $real")
    #println("Appx Values: $estimate")
    #println()
#    sumError = 0.
#    for i=1:length(z)
#        sumError += abs2(real[i]-estimate[i])
#    end
#    totalError = sqrt(sumError/stepsZeroToOne)
    totalError = sqrt(dot(weights,abs2.(real-estimate)))
    return totalError
end

#computeFirstBaroclinicError(15, firstBaroclinic, 10)

errors = zeros(20)
nValues = 3:22;
for i =nValues
    #println("N = $i")
    errors[i-2] = computeFirstBaroclinicError(i, firstBaroclinic, 201, S_constant)
    #println()
end
#plot(nValues,errors)
loglog(nValues[2:2:end], errors[2:2:end])

# Finite Difference Analysis

In [ ]:
function z_elements(size)
    stepSize = 1/size
    elements = zeros(size)
    elements[1] = stepSize/2
    for i = 2:size
        elements[i] = elements[i-1]+stepSize;
    end
    return elements
end

function U_fd(size, u_function)
    z_k = z_elements(size)
    fullMatrix = Diagonal(u_function(z_k))
    return fullMatrix
end

function L_fd(size, S)
    deltaZ = 1/size
    fullMatrix = zeros(size, size)
    fullMatrix[1,1] = S(deltaZ)
    fullMatrix[1,2] = -S(deltaZ)
    fullMatrix[size,size] = S((size-1)*deltaZ)
    fullMatrix[size,size-1] = -S((size-1)*deltaZ)
    for i=2:size-1
        fullMatrix[i, i-1] = -S((i-1)*deltaZ)
        fullMatrix[i,i] = S((i-1)*deltaZ)+S(i*deltaZ)
        fullMatrix[i, i+1] = -S(i*deltaZ)
    end
    normalize = 1./(deltaZ^2)
    return fullMatrix*normalize
end

function Q_fd(size, u_function, L_matrix)
    zk = z_elements(size)
    u_matrix = u_function(zk)
    diagonals = L_matrix*u_matrix
    fullMatrix = Diagonal(diagonals)
    return fullMatrix
end

#z_elements(8)
#L = L_fd(5, z_linear)
#Q_fd(5,u_linear,L)

In [ ]:
function u_linear(value)
    return value
end

function growthRateEigenValues_fd(sizeOfMatrix, beta_value, ky, kxValues, S_function, u_function)
    U = U_fd(sizeOfMatrix, u_function)
    L = L_fd(sizeOfMatrix, S_function)
    Q = Q_fd(sizeOfMatrix, u_function, L)
    #println("FD Matrices Computed")
    
    steps = length(kxValues)
    
    growthRates = zeros(steps)
    waveSpeeds = zeros(steps)
    minWaveSpeeds = zeros(steps)
    eigenvectors = []
    for i=1:steps
        #println()
        kx = kxValues[i]
        lhs = U*(Diagonal(zeros(sizeOfMatrix)+(kx^2+ky^2))+L) - (Q+Diagonal(zeros(sizeOfMatrix)+beta_value))
        rhs = Diagonal(zeros(sizeOfMatrix)+(kx^2+ky^2))+L
        #println(lhs)
        eigenObject = eigfact(lhs, rhs)
        max_im, max_ind = findmax(imag(eigenObject[:values]))
        #println(eigenObject[:vectors])
        append!(eigenvectors, [eigenObject[:vectors][:,max_ind]])
        waveSpeeds[i] = real(eigenObject[:values][max_ind])
        minWaveSpeeds[i] = minimum(real(eigenObject[:values]))
        growthRates[i] = max_im*kx
    end
    return growthRates, eigenvectors, waveSpeeds, minWaveSpeeds
end

# Finite Difference - Eady Problem

In [ ]:
using PyPlot
matrixSize = 7

kxStart = 0.1
kxFinish = 10.
steps = 1000
beta = 0
ky = 0

kxs = linspace(kxStart, kxFinish, steps)
growthRates, vectors, w_speeds, w_minSpeeds = growthRateEigenValues_fd(matrixSize, beta, ky, kxs, S_constant, u_linear)
plot(kxs, growthRates)

# Comparison Finite Difference to Galerkin

In [ ]:
using PyPlot
ioff()

nodePoints = 200 # redefining for ease of alteration

first_coefficient = 0.5
steps = 1
beta = 0
ky = 0
kxs = 1.61

nValues = 5:30
growthRates_galerkin = zeros(length(nValues))
growthRates_fd = zeros(length(nValues))

matrixSize = 30
eigenValuesAndVectors = getCompleteStabilityEigenObjectsOfKx(matrixSize, beta, ky, kxs, S_constant, dq, dz, dy_vartheta_bar(S_constant, dz, 1.), dy_vartheta_bar(S_constant, dz, 0.), first_coefficient)
growthRate_g, vectors, w_speeds, w_minSpeeds = getStabilityValues(eigenValuesAndVectors, kxs)

bestValues = zeros(length(nValues))+growthRate_g[1]

for i = 1:length(nValues)
    nV = nValues[i]
    matrixSize = nV
    println("Working on $nV")
    eigenValuesAndVectors = getCompleteStabilityEigenObjectsOfKx(matrixSize, beta, ky, kxs, S_constant, dq, dz, dy_vartheta_bar(S_constant, dz, 1.), dy_vartheta_bar(S_constant, dz, 0.), first_coefficient)
    growthRate_g, vectors, w_speeds, w_minSpeeds = getStabilityValues(eigenValuesAndVectors, kxs)
    growthRates_galerkin[i] = growthRate_g[1]
    
    matrixSize = nV + 2
    growthRate_fd, vectors_fd, w_speeds_fd, w_minSpeeds_fd = growthRateEigenValues_fd(matrixSize, beta, ky, kxs, S_constant, u_linear)
    growthRates_fd[i] = growthRate_fd[1]
end

loglog(nValues, abs.(growthRates_galerkin-bestValues), label="Galerkin")
loglog(nValues, abs.(growthRates_fd-bestValues), label="FD")
legend(loc="upper right")

# Finite Difference - Ocean Charney

In [ ]:
using PyPlot

matrixSize = 128

kxStart = 0.1
kxFinish = 50.
steps = 200
beta = 1
ky = 0

kxs = linspace(kxStart, kxFinish, steps)
growthRates_fd, vectors, w_speeds, w_minSpeeds = growthRateEigenValues_fd(matrixSize, beta, ky, kxs, S_OC, u_OC)
fig, axes = subplots(2,1)
ax = axes[1,1]
ax[:plot](kxs, growthRates_fd, label="N = $matrixSize")
ax[:legend](loc="upper right")
ax = axes[2,1]
ax[:plot](kxs, w_speeds, "b-")
ax[:plot](kxs, w_minSpeeds, "b-")

# Finite Difference - Phillips

In [ ]:
using PyPlot

matrixSize = 64

kxStart = .1
kxFinish = 4.
steps = 400
beta = 3.1
ky = 0

kxs = linspace(kxStart, kxFinish, steps)
growthRates_fd, vectors, w_speeds, w_minSpeeds = growthRateEigenValues_fd(matrixSize, beta, ky, kxs, S_Phillips, u_Phillips)
plot(kxs,growthRates_fd)

In [ ]:
nValues = 5:30
bestValue_fd = findmax(growthRates_fd)
growthRates_forN_fd = zeros(length(nValues))
for i =1:length(nValues)
    kxValue = 3.0
    matrixSize = nValues[i]
    println("Working on $matrixSize")
    growthRates_fd, vectors, w_speeds, w_minSpeeds = growthRateEigenValues_fd(matrixSize, beta, ky, kxValue, S_Phillips, u_Phillips)
    growthRates_forN_fd[i] = growthRates_fd[1]
end

loglog(nValues, abs.(growthRates_forN_fd-bestValue_fd[1]), label="Growth Rate Approach")
legend(loc="upper right")

In [ ]:

fig, axes = subplots(1,3)
ax = axes[1]
ax[:plot](kxs, growthRates_50, label="Growth Rates at N = 50")
ax = axes[2]
ax[:semilogy](nValues, abs.(growthRates_forN-bestValue[1]))
ax = axes[3]
ax[:loglog](nValues, abs.(growthRates_forN_fd-bestValue_fd[1]))